<center>
<img src="https://raw.githubusercontent.com/MLGlobalHealth/StatML4PopHealth/main/practicals/resources/logos/imperial.png" width="250" vspace="8"/>
<img src="https://raw.githubusercontent.com/MLGlobalHealth/StatML4PopHealth/main/practicals/resources/logos/mlgh.png" width="220" hspace="50" vspace="5"/>
<img src="https://raw.githubusercontent.com/MLGlobalHealth/StatML4PopHealth/main/practicals/resources/logos/ammi.png" width="190"/>

<font size="6">Modern Statistics and Machine Learning <br>for Population Health in Africa </font>

<font size="4">24th - 28th March 2025</font>

</center>

# Compartmental Models in Stan
### Ettie Unwin

The first part of this tutorial is based on a paper by [Ginsztajn et al.](https://onlinelibrary.wiley.com/doi/full/10.1002/sim.9164). This work is licensed under a [Creative Commons Attribution-NonCommercial-NoDerivatives 4.0 International License](https://creativecommons.org/licenses/by-nc-nd/4.0/).

In this tutorial, we demonstrate how to formulate, fit, and diagnose compartmental model like the ones we considered this morning in Stan using the [cmdstanpy](https://mc-stan.org/cmdstanpy/) interface. (See yesterday's notes for why we are interested in using Stan). We consider two different types of routinely collected data to fit our models to: prevalence data (number / proportion of infected people in a population) and incidence data (newly infected people in a population).

In this example, we examine an outbreak of influenza A (H1N1) in 1978 at a British boarding school. The data consists of the daily number of students in bed (prevalence data), spanning over a time interval of 14 days. There were 763 male students who were mostly full boarders and 512 of them became ill. The outbreak lasted from the 22nd of January to the 4th of February. It is reported that one infected boy started the epidemic, which spread rapidly in the relatively closed community of the boarding school. The data are freely available in the R package outbreaks, maintained as part of the [R Epidemics Consortium](http://www.repidemicsconsortium.org/), and is a great resource for trying to fit different models yourself after the course. This is only available as an R package so we have saved the data as a .json file to use for this practical. Since there was no publicly available incidence data associated with this outbreak, we provide some simulated data created to mimic the behaviour of the outbreak.

## Install cmdstanpy
Like the other practicals, we will start by install cmdstanpy for this notebook.

In [ ]:
!curl -O "https://raw.githubusercontent.com/MLGlobalHealth/StatML4PopHealth/main/practicals/resources/scripts/utilities.py"

from utilities import custom_install_cmdstan, test_cmdstan_installation

custom_install_cmdstan()

# if we want to test the installation, we can run the bernoulli hello world example
test_cmdstan_installation()

## Exploring the data

First we will load in all the images and data that we will need for this practical.


In [ ]:
import subprocess

resource_urls= [
    # data
    "https://raw.githubusercontent.com/MLGlobalHealth/StatML4PopHealth/main/practicals/day4/practical5/data/flu_data.csv",
    "https://raw.githubusercontent.com/MLGlobalHealth/StatML4PopHealth/main/practicals/day4/practical5/data/incidence.csv",
    # # Stan models
    "https://raw.githubusercontent.com/MLGlobalHealth/StatML4PopHealth/main/practicals/day4/practical5/stan_models/seir_incidence.stan",
    "https://raw.githubusercontent.com/MLGlobalHealth/StatML4PopHealth/main/practicals/day4/practical5/stan_models/seir_prevalence.stan",
    "https://raw.githubusercontent.com/MLGlobalHealth/StatML4PopHealth/main/practicals/day4/practical5/stan_models/sir_model.stan",
    "https://raw.githubusercontent.com/MLGlobalHealth/StatML4PopHealth/main/practicals/day4/practical5/stan_models/sir_model_prior.stan"
     ]

# Note this will overwrite existing files, remove -O if that is not desired
for url in resource_urls:
   subprocess.run(["curl", "-O", url], check=True)

!ls

In [ ]:
!cat seir_incidence.stan

!cat seir_incidence.stan

Now we can use these files.  The first one we will explore is the flu data.

In [ ]:
# Load in the data using pandas
import pandas as pd
df = pd.read_csv('flu_data.csv')

# Convert the dates to datetime format
df.Date = pd.to_datetime(df.date)

# Use date as index
df = df.set_index("date")

print(df)

From this we can see that we have a two week timeseries, as expected, with dates, numbers of students in bed and the number who are convalescing.

Q1) Make a plot of the timeseries.

In [ ]:
import matplotlib.pyplot as plt

fig, ax1 = plt.subplots()
ax1.grid(True, which='both', color = 'k', alpha = 0.1)
ax1.set_ylabel('Number of children')
ax1.plot(df.in_bed, color = 'tab:red', linewidth= 1, label = 'In bed')
ax1.plot(df.convalescent, color = 'tab:blue', linewidth= 1, label = "Convalescent")
ax1.set_facecolor('white')
ax1.legend(loc = "upper right")
ax1.tick_params(axis='x', labelrotation=90)


## Fitting an SIR (transmission) model to prevalence data
The simplest model we can consider is the Susceptible-Infected-Recovered (SIR) model. This splits the population in three time-dependent compartments: the susceptible ($S$), the infected (and infectious) ($I$), and the recovered (and not infectious) compartments ($R$). When a susceptible individual comes into contact with an infectious individual, the former can become infected for some time, and then recover and become immune.  

<img src=" https://raw.githubusercontent.com/MLGlobalHealth/StatML4PopHealth/main/practicals/day4/practical5/images/sir.png">

From Figure 1, the rate of progression between the S and I compartments is $\beta * I / N$ where $\beta$ is the contact rate, $I$ is the number of infected people and $N$ is the total population. The rate of progression between the $I$ and $R$ compartments is $\sigma$ the recovery rate or 1 divided by the duration of infection.



Q2) Define the equations for an SIR model.



$$\frac{dS}{dt} = -\beta \frac{I}{N} S$$
$$\frac{dI}{dt} = \beta \frac{I}{N} - \sigma I $$
$$\frac{dR}{dt} = \sigma I $$



Q3) What are the assumptions of this model?

- homogeneous mixing
- no births or deaths
- permanent immunity

## Defining the statistical model
We now define the statistical inference model that we will use to fit our tranmission model to data. Here we introduce the likelihood or sampling distribution for our model
$$p(y|\theta),$$
which tells us, given model parameters $\theta$, how to generate data $y$. In a Bayesian framework, the set of plausible values is characterised by the posterior distribution,
$$
p(\theta | y).
$$

Bayes’ rule teaches us that
$$
p(\theta | y) \sim p(y | \theta) p(\theta)
$$
where $p(\theta | y)$ is our posterior distribution and this is proportional to $p(y|\theta)$ our likelihood multiplied by $p(θ)$ our prior distribution. The prior encodes information or our belief about the parameters we have before observing the data.

### The likelihood
This is the part of the model that we used to link the model estimates of the number of infected students, $I_{ODE}(t)$, for given parameters ($\beta$ and $\sigma$) to the observed data, i.e the number of students in bed, $I_{obs}(t)$. We choose to model the number of students in bed with a count distribution – the Negative Binomial. This distribution allows us to use $I_{ODE}(t)$ as the expected value and account for over-dispersion, through the parameter $\phi$:
$$
I_{obs}(t)∼NegBin(I_{ODE}(t), \phi)
$$
Therefore our likelihood is $p(y∣\theta)$, with $\theta = (\beta, \sigma, \phi)$ as the parameters of the model.


### Priors
We need to specify a prior for each of our three parameter $\theta = (\beta, \sigma, \phi)$. One advantage of the Bayesian approach is that we can formally incorporate prior knowledge about the parameters into the model. We can change this prior if more information becomes available, constraining our parameter estimation more tightly or, on the contrary, increasing its variance. Choosing priors is one of the tricky parts of Bayesian modelling but comes with practice.


### Predictions and derived quantities
Once we fit the model and obtain a posterior distribution for $\theta$, we can derive additional quantities of interests. The posterior distribution of predictions:
$$
p(y_{pred}∣y)=\int p(y_{pred}|\theta) p(\theta | y)d\theta.
$$

## Coding the model in Stan

We will need to import the following packages to fit the compartmental model in Stan.


In [ ]:
# Load packages used in this installation
import os
import json
import shutil
import urllib.request

from cmdstanpy import CmdStanModel

Now we need to install CmdStanPy.

Step 1: install CmdStanPy

### Coding the ODE
An ODE takes the form
$$
\frac{dz}{dt}=f(t,z)
$$
where $z$ are the states of the compartmental model ($z=(S,I,R)$ for our first example) and $t$ is time. We also need to provide initial conditions $z_0$ at $t_0$ and the times, $τ$, at which we evaluate the solution.

To specify an ODE in Stan, we first code $f$ in the functions block. This function must observe a strict signature:

```python
vector f(real time, vector state, ...)
```

where

* time, $t$;
* state, the different compartment, $z$;
* ..., optional arguments for the different parameters in the model.

In our example, the ODEs for the SIR model is defined with the following structure.

```python
functions {
  vector sir(real t,
             vector y,
             real beta,
             real sigma,
             real N) {

      vector[3] dydt;

      real S = y[1];
      real I = y[2];
      real R = y[3];
      
      dydt[1] =  S equation;
      dydt[2] =  I equation;
      dydt[3] =  R equation;
      
      return dydt;
  }
}



Q4) Fill in the equations for the differential equations for S, I and R:

```python
functions {
  vector sir(real t,
             vector y,
             real beta,
             real sigma,
             real N) {

      vector[3] dydt;

      real S = y[1];
      real I = y[2];
      real R = y[3];

      dydt[1] =  fill in here;
      dydt[2] =  fill in here;
      dydt[3] =  fill in here;

      return dydt;
  }
}
```

We evaluate the solution numerically by using one of Stan’s numerical integrators.  Here we choose the Runge-Kutta 4th / 5th order and a call to the integrator looks as follows:

```python
y = ode_rk45(sir, y0, t0, t, beta, sigma, N);;
```
where

* sir, the name of the function that returns the derivatives that we wrote earlier;
* y0, the initial condition;
* t0, the time of the initial condition;
* t, the times at which we require the solution to be evaluated;
* beta, the transmission parameter;
* sigma, the recovery rate;
* N, the number of people in our population.

We now have all the ingredients to solve our ODE.


### Building the model
We next code the model in Stan, working through the following coding blocks.

A) We first need the function block from above that specifies the SIR model.  

B) Fixed data is declared in the data block.  This is things like the initial conditions, times and number of days.
```python
data {
  int n_days;
  vector[3] y0;
  real t0;
  array[n_days] real t;
  int N;
  array[n_days] int cases;
}
```

C) We next declare the model parameters. If you want some parameter to be bounded, and it is not already guaranteed by it's prior, you need to specify <lower=a, upper=b> when declaring this parameter. Note that this is how you put a truncated prior distribution on a parameter and here we choose to define a prior on the inverse of $\phi$.  
```python
parameters {
  real<lower=0> sigma;
  real<lower=0> beta;
  real<lower=0> phi_inv;
}
```

D) And then transforms of the parameters
```python
transformed parameters{
  real y[n_days, 3];
  real phi = 1. / phi_inv;
  
  y = integrate_ode_rk45(sir, y0, t0, t, beta, sigma, N);
}
```

E) Our model consists of the prior and likelihood as described above. Here we assume some priors for our parameters and define our likelihood.
```python
model {
  //priors
  beta ~ normal(4, 2); //truncated at 0
  sigma ~ normal(0.8, 0.3); //truncated at 0
  phi_inv ~ exponential(5);
  
  //likelihood
  cases ~ neg_binomial_2(y[:, 2], phi);
}
```

F) Finally untangled from the inference, we predict the number of cases, basic reproduction number and recovery time in the generated quantities block:

```python
generated quantities {
  real R0 = beta / sigma;
  real recovery_time = 1 / sigma;

  array[n_days] real pred_cases;
  pred_cases = neg_binomial_2_rng(y[:,2], phi);
}
```


### Fitting the model in python

Here we write the code to fit the model in python to the number of children in bed.  You need to copy all the model parts (functions, data, parameters, transformed parameters, model and generated quantities) from the above section into a file called  ```sir_model.stan``` in a directory called ```stan_models```.

Next we create the data struture to fit our model to.

In [ ]:
import numpy as np

# Set the population size
N = 763

# Define initial conditions
i0 = 1
s0 = N - i0
r0 = 0
y0 = [s0, i0, r0]

# Make the data struture
stan_data = {
    "n_days": len(df.in_bed),
    "y0": y0,
    "t0": 0,
    "t": np.arange(1, len(df.in_bed)+1),
    "N": N,
    "cases": df.in_bed
}

Then we compile our stan model.

In [ ]:
sir_model = CmdStanModel(stan_file = 'sir_model.stan')


Finally we run MCMC. For this problem, it suffices to use Stan’s defaults. Note that, as is standard practice, we run 4 Markov chains.

In [ ]:
fit_sir_model = sir_model.sample(data = stan_data,
                                 iter_sampling = 2000,
                                 chains = 4,
                                 seed = 0)

We notice this prints lots of debugging statements to the display. This can be limited by the following command.

In [ ]:
import logging

cmdstanpy_logger = logging.getLogger("cmdstanpy")
cmdstanpy_logger.disabled = True

In [ ]:
fit_sir_model = sir_model.sample(data = stan_data,
                                 iter_sampling = 2000,
                                 chains = 4,
                                 seed = 0)

## Checking the model

When developing a model it is good to check the model is performing as you expect it.

### Inference
A good place to start is with a summary table of the results, which displays the posterior mean, standard error, quantiles, and some useful diagnostics. Since there are lots of parmeters in the model we will consider 'beta', 'sigma' and 'recovery time' only here

In [ ]:
sir_summary = fit_sir_model.summary()
sir_summary.filter(items=("sigma", "beta", "recovery_time"), axis=0)

Stan gives us lots of information to evaluate whether the inference is reliable:

* During sampling, warnings can tell us if something is wrong (here we have no warnings but you should check yours).
* In the summary table, several quantities are available to check inference.
  - Here we note that $\hat R$ is close to 1 (< 1.001), indicating the 4 Markov chains are in close agreement with one another.
  - The effective samples size, ESS_bulk and ESS_tail, are large, which means the Markov chains were able to cohesively explore the parameter space.
  - Conversely, large $\hat{R}$ and low effective sample size would indicate that the Markov chains are poorly mixing.
  
Apart from fixing coding errors, improving the mixing of the Markov chains almost always requires tweaking the model specification, for example with a reparameterisation or stronger priors.

We can use the arivz package to  plot the marginal posterior densities for each chain, which enables us to confirm the Markov chains are in agreement with one another, and the trace plots to explore how well our chains have mixed.

In [ ]:
import arviz as az

az.style.use("arviz-doc")
az.plot_trace(fit_sir_model,
              var_names = ["beta", "sigma"])

### Model
Now that we trust our inference, we can check the utility of our model. Utility is problem specific and can include the precise estimation of a quantity or predicting future behaviors. In general, it is good to check if our model, once fitted, produces simulations that are consistent with the observed data. This is the idea behind posterior predictive checks.

We sample predictions, $y_{pred}$, from $p(y_{pred}∣y)$ and use these samples to construct a fitted curve for students in bed, together with the uncertainty (90% interval, meaning observed data is expected to fall outside of this interval one in ten times). This posterior predictive check allows us to verify if the model captures the structure of the data. Here we see that the model gives a satisfying fit to the data, and that the model uncertainty is able to capture the variation of the data.

In [ ]:
post_pred = pd.DataFrame(sir_summary.filter(like="pred_cases", axis=0))
post_pred["t"] = range(1, 15)
post_pred["cases"] = list(df.in_bed)

post_pred = post_pred.set_index("t")
post_pred

In [ ]:
fig, ax1 = plt.subplots()
ax1.grid(True, which='both', color = 'k', alpha = 0.1)
ax1.set_ylabel('Number of children in bed')
ax1.set_xlabel('Time')
ax1.scatter(x = post_pred.index, y = post_pred.cases, color = 'tab:red', label = "Cases")
ax1.plot(post_pred.Mean, color = 'tab:blue', linewidth= 1, label = "Mean estimated cases")
ax1.fill_between(x = post_pred.index,  y1 = post_pred['5%'], y2 = post_pred['95%'], alpha=0.2)
ax1.set_facecolor('white')
ax1.legend(loc = "upper right")

In [ ]:
infected = sir_summary.filter(items=("y[1,2]", "y[2,2]", "y[3,2]", "y[4,2]", "y[5,2]",
                                     "y[6,2]", "y[7,2]", "y[8,2]", "y[9,2]", "y[10,2]",
                                     "y[11,2]", "y[12,2]", "y[13,2]", "y[14,2]",), axis=0)

infected["t"] = range(1, 15)
infected = infected.set_index("t")

fig, ax1 = plt.subplots()
ax1.grid(True, which='both', color = 'k', alpha = 0.1)
ax1.set_ylabel('Number of infected children')
ax1.set_xlabel('Time')
ax1.plot(infected.Mean, color = 'tab:blue', linewidth= 1, label = "Mean estimated number of infected")
ax1.fill_between(x = infected.index,  y1 = infected['5%'], y2 = infected['95%'], alpha=0.2)
ax1.set_facecolor('white')

### Priors
We can check if our priors are sound by computing the a priori probability of various epidemiological parameters of interest. For instance for influenza, we know that the recovery time is approximately 1 week. We want priors that allow for every reasonable configurations of the data but exclude absurd scenarios, per our domain expertise. To check if our priors fulfill this role, we can do a prior predictive check.

To conduct a prior predictive check, we take the same model as before, put the parameters of interest in the generated_quantities code block, and remove the likelihood term from the model. Without the likelihood, the parameters are not fitted to the data and are thus sampled from their prior distribution. The Stan code is thus the same as the final Stan code, without the ```cases ~ neg_binomial_2(y[:,2], phi);``` line.

We can now compile the model without the likelihood term.  

* You need to create a new model file called ```sir_model_prior.stan``` by removing the likelihood term from your model and saving it in your drive.
* You also need to change ``` pred_cases = neg_binomial_2_rng(y[:, 2], phi);``` to ``` pred_cases = neg_binomial_2_rng(y[:, 2] + 1e-5, phi);``` to ensure all your values are defined.

The following code will mount your drive to this notebook and load in the model:

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

#stan_file_prior = 'drive/MyDrive/sir_model_prior.stan'
sir_model_prior = CmdStanModel(stan_file = 'sir_model_prior.stan')


Then we can sample from it.

In [ ]:
prior_check = sir_model_prior.sample(data = stan_data,
                                     iter_sampling = 2000,
                                     chains = 4,
                                     seed = 0)

This gives us samples from the a priori distribution of parameters, which we can visualize. Here we show the distribution of the log of the recovery time, with the red bars showing loose bounds on the recovery time (1/2 day and 30 days). We observe that most of the probability mass is between the red bars but we still allow more extreme values, meaning our posterior can concentrate outside the bars, if the data warrants it.


In [ ]:
recovery_time = prior_check.stan_variable("recovery_time")
ax = pd.DataFrame(recovery_time).plot(kind='density', logx=True)
ax.axvline(x=0.5)
ax.axvline(x=30)


We thus see that these distributions are coherent with domain knowledge. See [here](https://github.com/stan-dev/stan/wiki/Prior-Choice-Recommendations) for more recommendations on prior choice.

We can also plot trajectories of infection according to the prior, that is the number of infected people at each time according to prior distributions of parameters. The dashed red line shows the total population of children.


In [ ]:
infection_draws = prior_check.draws_pd("y")
lines = plt.plot(range(1,15),
                 infection_draws.iloc[:, [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]].T,
                 color = "k", alpha = 0.1)
plt.axhline(y=763, color='r', linestyle='--')
plt.xlabel('Time')
plt.ylabel('Number of infected children')
plt.show()


And the median (blue line) and 90% interval of the a priori number of student in bed (i.e the observed number of infected students).

In [ ]:
prior_pred_cases = pd.DataFrame(prior_check.summary().filter(like="pred_cases", axis=0))
prior_pred_cases["t"] = range(1, 15)
prior_pred_cases = prior_pred_cases.set_index("t")

fig, ax1 = plt.subplots()
ax1.grid(True, which='both', color = 'k', alpha = 0.1)
ax1.set_ylabel('Number of children in bed')
ax1.set_xlabel('Time')
ax1.plot(prior_pred_cases.Mean, color = 'tab:blue', linewidth= 1, label = "Mean estimated cases")
ax1.fill_between(x = prior_pred_cases.index,  y1 = prior_pred_cases['5%'], y2 = prior_pred_cases['95%'], alpha=0.2)
ax1.set_facecolor('white')
ax1.legend(loc = "center right")
ax1.axhline(y=763, color='r', linestyle='--')


It seems that most trajectories are reasonable and quite diverse. Still, some of the curves look a little bit funky and suggest we could refine our priors and make them more informative, although it may not be needed here.

Typically, we can get away with priors that do not capture all our a priori knowledge, provided the data is informative enough. However when dealing with complicated models and relatively sparse data, we usually need well constructed priors to regularize our estimates and avoid non-identifiability.


## Fitting an SEIR model to prevalence data

A person who is infected with flu is not immediately infectious so it makes sense to include an "Exposed" compartment in our model. This should improve the fit to our model (although we don't evaluate that here in this practical).

<img src=" https://raw.githubusercontent.com/MLGlobalHealth/StatML4PopHealth/main/practicals/day4/practical5/images/seir.png">

The notation in Figure 2 is similar to the SIR model but we have a new parameter $\gamma$ which is the rate of progression from the exposed to infected compartment.  It is equal to 1 divided by the incubation period.

Q4) What are the equations for an SEIR model?

$$\frac{dS}{dt} = -\beta \frac{I}{N} S$$
$$\frac{dE}{dt} = \beta \frac{I}{N} - \gamma E $$
$$\frac{dI}{dt} = \gamma E - \sigma I $$
$$\frac{dR}{dt} = \sigma I $$


Q5) Edit the stan model file to turn the SIR model in an SEIR model. Save this in the ```stan_model``` directory under a new file called ```seir_prevalence.stan```.  Since we have a new parameter in our model $\gamma$ we need to define a prior for this.  From the literature the incubation period for flu is around 2 days so a prior for gamma of $N(0.5, 0.25)$ would be suitable.  

Hint:  You will need to change the following parts of the model.

- The functions block to encode the new ODE model,
- The data block so that y0 has dimensions of 4,
- The parameter block so the gamma parameter is added,
- The model block needs the prior for gamma adding and,
- The generated quantity block can be used to calculate the incubation period (1/gamma).

Similar to before, the SEIR model can be run as follows. Note we also need to add an initial condition for our exposed compartment.  

In [ ]:
# Set the population size
N = 763

# Define initial conditions
i0 = 1
e0 = 0
s0 = N - i0
r0 = 0
y0 = [s0, e0, i0, r0]

# Make the data struture
seir_data = {
    "n_days": len(df.in_bed),
    "y0": y0,
    "t0": 0,
    "t": np.arange(1, len(df.in_bed)+1),
    "N": N,
    "cases": df.in_bed
}

seir_model = CmdStanModel(stan_file = 'seir_prevalence.stan')

fit_seir_model = seir_model.sample(data = seir_data,
                                   iter_sampling = 2000,
                                   chains = 4,
                                   seed = 0)


Q6) Print a summary for your results:

In [ ]:
seir_summary = fit_seir_model.summary()
seir_summary.filter(items=("sigma", "beta", "recovery_time", "gamma", "incubation_period"), axis=0)

Q7) Produce plots for our sample predictions of the number of students in bed.


In [ ]:
post_pred = pd.DataFrame(seir_summary.filter(like="pred_cases", axis=0))
post_pred["t"] = range(1, 15)
post_pred["cases"] = list(df.in_bed)

post_pred = post_pred.set_index("t")
post_pred

fig, ax1 = plt.subplots()
ax1.grid(True, which='both', color = 'k', alpha = 0.1)
ax1.set_ylabel('Number of children in bed')
ax1.set_xlabel('Time')
ax1.scatter(x = post_pred.index, y = post_pred.cases, color = 'tab:red', label = "Cases")
ax1.plot(post_pred.Mean, color = 'tab:blue', linewidth= 1, label = "Mean estimated cases")
ax1.fill_between(x = post_pred.index,  y1 = post_pred['5%'], y2 = post_pred['95%'], alpha=0.2)
ax1.set_facecolor('white')
ax1.legend(loc = "upper right")

## Fitting an SEIR model to incidence data

The number of infected people in a population is not the only type of data collected during an outbreak. Often we get a line list or a list of the times at which people were infected (or showed symptoms). We have simulated some incidence data for a flu like outbreak with similar parameters to the school data and this can be found in the file ```incidence_seir.csv```. (Do speak to one of us if you are interested in how we did this!).


In [ ]:
# Read in incidence data

# Load in the data using pandas
import pandas as pd
df = pd.read_csv('incidence.csv')

print(df)


We now need to change our model to fit the incidence from our model to the data rather than the prevalence. First we need to calculate the incidence, which is equal to the number of people leaving the exposed compartment at each time step.  

Q8) Make a new copy of our SEIR model by re-saving ```seir_prevalence.stan``` as ```seir_incidence.stan```. The incidence can be calculated by first adding an ODE for cumumlative incidence in the functions block.

```python
dydt[5] = gamma * E
```
Remember you will need to change the dimensions of dydt, the output of solving your differential equations y and create a new initial condition.  Then you will need to create a for loop that calculates incidence.

```python
incidence[1] = y[1, 5] - 0
for (i in 2:n_days)
  incidence[i] = y[i, 5] - y[i-1, 5];
```

Then we need to update the likelihood in the model block to the following since we are now saying that our incidence is negatively binomially distributed instead of the total number of cases.
```python
//sampling distribution
cases ~ neg_binomial_2(incidence, phi);
```

Q9) Use similar code to before to fit our model and print a summary of our parameters.

In [ ]:
import numpy as np

# Set the population size
N = 763

# Define initial conditions
i0 = 1
e0 = 0
s0 = N - i0
r0 = 0
c0 = 0
y0 = [s0, e0, i0, r0, c0]

# Make the data struture
seir_data = {
    "n_days": len(df.x),
    "y0": y0,
    "t0": 0,
    "t": np.arange(1, len(df.x)+1),
    "N": N,
    "cases": df.x
}

seir_incidence_model = CmdStanModel(stan_file = 'seir_incidence.stan')

fit_seir_incidence_model = seir_incidence_model.sample(data = seir_data,
                                                       iter_sampling = 2000,
                                                       chains = 4,
                                                       seed = 0)

seir_incidence_summary = fit_seir_incidence_model.summary()
seir_incidence_summary.filter(items=("sigma", "beta", "recovery_time", "gamma", "incubation_period"), axis=0)

Q10) Plot your data and model fit.

In [ ]:
import matplotlib.pyplot as plt

post_pred = pd.DataFrame(seir_incidence_summary.filter(like="pred_incidence", axis=0))
post_pred["t"] = range(1, 14)
post_pred["incidence"] = list(df.x)

post_pred = post_pred.set_index("t")

fig, ax1 = plt.subplots()
ax1.grid(True, which='both', color = 'k', alpha = 0.1)
ax1.set_ylabel('Incidence')
ax1.set_xlabel('Time')
ax1.scatter(x = post_pred.index, y = post_pred.incidence, color = 'tab:red', label = "Incidence")
ax1.plot(post_pred.Mean, color = 'tab:blue', linewidth= 1, label = "Mean estimated incidence")
ax1.fill_between(x = post_pred.index,  y1 = post_pred['5%'], y2 = post_pred['95%'], alpha=0.2)
ax1.set_facecolor('white')
ax1.legend(loc = "upper right")

Q11) Explore what impact changing your priors has on the model fit.


## Extension material

Now you have fit simple SIR and SEIR model to some data based on influenser, we will consider using all the material you have learned today to design and fit a compartmental model of Ebola to the line list incidence data found in the supplementary material of [Garske (2017)](https://royalsocietypublishing.org/doi/10.1098/rstb.2016.0308).

Suggested workflow:

1) Download and investigate the [data](https://royalsocietypublishing.org/action/downloadSupplement?doi=10.1098%2Frstb.2016.0308&file=rstb20160308supp1.csv)

2) Do a quick literature review about how Ebola is spread.  A few good resources: [Drake et al. (2015)](https://pmc.ncbi.nlm.nih.gov/articles/PMC4517740/), [WHO fact sheet](https://www.who.int/news-room/fact-sheets/detail/ebola-virus-disease) and [Nash et al. (2024)](https://www.thelancet.com/article/S1473-3099(24)00374-8/abstract).

3) Design a flow chart for Ebola transmission using pen and paper.

4) Write the equations for this model.

5) Code them using python and stan.

6) Investigate what priors make sense.